In [1]:
from PIL import Image, ImageFilter, ImageOps
import numpy as np
import requests
from io import BytesIO
from skimage import transform as tf

def fade_frame(img, sides=20, bottom=40):
    img = img.filter(ImageFilter.BoxBlur(sides)).crop((sides, sides, img.width - sides, img.height - sides))
    img = img.filter(ImageFilter.BoxBlur(bottom)).crop((0, bottom, img.width, img.height - bottom))
    return img

response = requests.get('https://i.stack.imgur.com/BM8Bu.png')
doge_img = Image.open(BytesIO(response.content))
doge_img = ImageOps.expand(doge_img, (0, 0, 0, 100), fill=(255, 255, 255))
doge_img = doge_img.resize((int(doge_img.width * 0.5), int(doge_img.height * 0.5)))
doge_img = fade_frame(doge_img)

scales = [1/5, 1/3]
transfs = [np.eye(2) / scale for scale in scales]
shifts = np.array([[0.26, .98], [0.65, .95]]) / np.array(scales)[:, None]

doges_raw = []
for scaling in np.arange(1, 3, 0.1):
    resized_img = doge_img.resize((int(doge_img.width * scaling), int(doge_img.height * scaling)))
    for transf, shift in zip(transfs, shifts):
        tform = tf.AffineTransform(matrix=transf)
        shifted_img = tf.warp(np.array(resized_img), (tform + tf.AffineTransform(translation=-shift)).inverse)
        doges_raw.append(Image.fromarray((shifted_img * 255).astype(np.uint8)))

ModuleNotFoundError: No module named 'skimage'